<a href="https://colab.research.google.com/github/rraulison/olx/blob/main/OLX_scrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd_chrome = webdriver.Chrome(options=chrome_options)

In [ ]:

def dados(url):

  wd_chrome.get(url)
  iframe = wd_chrome.find_element(By.ID, value='column-main-content')
  iframe = iframe.find_element(By.XPATH,'//*[@id="ad-list"]')

  for i in range(1,60):
    try:
      modelo.append(iframe.find_element(By.XPATH,'//*[@id="ad-list"]/li['+str(i)+']/div/a/div/div[2]/div[1]/div[1]/div[1]/div/h2').text)
      preco.append(iframe.find_element(By.XPATH,'//*[@id="ad-list"]/li['+str(i)+']/div/a/div/div[2]/div[1]/div[1]/div[2]/div/div/div/span').text.replace('R$ ',''))
      km.append(iframe.find_element(By.XPATH,'//*[@id="ad-list"]/li['+str(i)+']/div/a/div/div[2]/div[1]/div[2]/div/div/div[2]/div/div[1]/span').text.replace(' km',''))
      ano.append(int(iframe.find_element(By.XPATH,'//*[@id="ad-list"]/li['+str(i)+']/div/a/div/div[2]/div[1]/div[2]/div/div/div[2]/div/div[2]/span').text))
      try:
        cambio.append(iframe.find_element(By.XPATH,'//*[@id="ad-list"]/li['+str(i)+']/div/a/div/div[2]/div[1]/div[2]/div/div/div[2]/div/div[3]/span').text)
      except NoSuchElementException:
        cambio.append(None)

    except:
      continue
 
  return {'modelo':modelo, 'preco':preco, 'mil kms':km, 'ano':ano, 'cambio':cambio}

In [ ]:
modelo,preco,ano,km,cambio =[],[],[],[],[]

for i in range(1,15): #numero de paginas consultadas (cada pagina tem cerca de 50 itens)
  url = 'https://www.olx.com.br/autos-e-pecas/carros-vans-e-utilitarios/estado-ce/fortaleza-e-regiao?ctp=8&ctp=7&ctp=1&me=100000&o='+str(i)+'&pe=39999&ps=16000&rs=29&sp=2' #pagina a ser consultada
  try: 
    dados(url)
  except NoSuchElementException:
    break

In [ ]:
preco = [int(x.replace('.', '')) for x in preco]
km = [int(float(x)) for x in km]
df = pd.DataFrame({'modelo':modelo, 'preco':preco, 'mil kms':km, 'ano':ano, 'cambio':cambio})
df

,modelo,preco,mil kms,ano,cambio
0,Carro,17000,105,2011,Gasolina
1,Peugeot passion 207,17500,0,2012,Manual
2,Pegeout 207 passion xs,17900,83,2011,Automático
3,Peugeot,18500,0,2012,Manual
4,Versa 1.6 com gnv particular (leia o anuncio),19000,90,2014,Manual
...,...,...,...,...,...
383,Hyundai HB20S - 2014 - Oportunidade! 20% abaix...,39990,95,2014,Manual
384,Fiat Palio Sporting 1.6 16V (Flex),39990,100,2013,Manual
385,"Vendo, troco e financiou Siena EL 1.4 ÚNICO DONO",39999,108,2014,Manual
386,"Vendo, troco e financiou GranSiena 1.4 2013",39999,106,2013,Manual
